In [31]:
import openml
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


import warnings
warnings.filterwarnings("ignore") # no warnings on the console

In [32]:
datasets = sorted(pd.unique(pd.read_csv("../results/results_randomsearch.csv", usecols=["openmlid"])["openmlid"]))
rows = []
for openmlid in tqdm(datasets):
    
    # load dataset object
    dataset = openml.datasets.get_dataset(int(openmlid))
    
    # analyze columns of data
    cnt_normalized = 0
    cnt_zero_mean = 0
    cnt_one_std = 0
    cnt_numerical = 0
    row = [
        openmlid,
        dataset.name[:20].replace("_", "\\_"),
        int(dataset.qualities["NumberOfInstances"]),
        len(dataset.features) - 1,
        int(dataset.qualities["NumberOfNumericFeatures"]),
        int(dataset.qualities["NumberOfClasses"]),
        str(int(dataset.qualities["MinorityClassPercentage"]) if dataset.qualities["MinorityClassPercentage"] != "NaN" in dataset.qualities else np.nan) + "\%",
        str(int(dataset.qualities["MajorityClassPercentage"])if dataset.qualities["MajorityClassPercentage"] != "NaN" in dataset.qualities else np.nan) + "\%"
    ]
    rows.append(row)
dfDatasets = pd.DataFrame(rows, columns=["openmlid", "name", "instances", "features", "numeric features", "classes", "min \%", "maj \%"]).sort_values("openmlid")

  0%|          | 0/77 [00:00<?, ?it/s]

In [35]:
dfDatasets["matrix size (in thds.)"] = (dfDatasets["instances"] * dfDatasets["features"] / 1000).astype(int)
print(dfDatasets.to_latex(index=False, escape=False))

\begin{tabular}{rlrrrrllr}
\toprule
 openmlid &                    name &  instances &  features &  numeric features &  classes & min \% & maj \% &  matrix size (in thds.) \\
\midrule
        3 &                kr-vs-kp &       3196 &        36 &                 0 &        2 &  nan\% &  nan\% &                     115 \\
        6 &                  letter &      20000 &        16 &                16 &       26 &  nan\% &  nan\% &                     320 \\
       11 &           balance-scale &        625 &         4 &                 4 &        3 &  nan\% &  nan\% &                       2 \\
       12 &           mfeat-factors &       2000 &       216 &               216 &       10 &  nan\% &  nan\% &                     432 \\
       23 &                     cmc &       1473 &         9 &                 2 &        3 &  nan\% &  nan\% &                      13 \\
       30 &             page-blocks &       5473 &        10 &                10 &        5 &  nan\% &  nan\% &          